In [10]:
import os
from smrt import make_snowpack, make_model, sensor, PSU, make_ice_column, sensor_list
from smrt.utils import dmrt_qms_legacy
import matplotlib.pyplot as plt
import utils
from tqdm import trange
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
from classes import evolving_pack

# Run a basic DMRT-QMS example

In [19]:
pc=0.2e-3
snowpack = make_snowpack([10],
                         microstructure_model="sticky_hard_spheres",
                         density=[300],
                         temperature=[265],
                         radius=pc,
                         stickiness = 0.15)

# create the sensor
theta = 55
radiometer = sensor.passive(19e9, theta)

# dmrt_qms_legacy.set_dmrt_qms_path('/home/robbie/DMRT-QMS')

mresult = dmrt_qms_legacy.run(radiometer, 
                              snowpack,
                              dmrt_qms_path = '/home/robbie/DMRT-QMS/')

print(mresult.TbV())

260.97029329156743


# Now Create a Typical Ice Pack

In [3]:
il = 10
thickness = np.array([1/il] * il) #ice is 1.5m thick
p_ex = np.array([1.0e-3] * (il)) #correlation length
temperature = np.linspace(273.15-20., 273.15 - 1.8, il) #temperature gradient in the ice from -20 deg C at top to freezing temperature of water at bottom (-1.8 deg C)
salinity = np.linspace(2., 10., il)*PSU #salinity profile ranging from salinity=2 at the top to salinity=10 at the bottom of the ice

# create a multi-year sea ice column with assumption of spherical brine inclusions (brine_inclusion_shape="spheres"), and 10% porosity:
ice_type = 'firstyear' # first-year or multi-year sea ice
porosity = 0.08 # ice porosity in fractions, [0..1]

ice_column = make_ice_column(ice_type=ice_type,
                            thickness=thickness,
                            temperature=temperature,
                            microstructure_model="exponential",
                            brine_inclusion_shape="random_needles", #brine_inclusion_shape can be "spheres", "random_needles" or "mix_spheres_needles"
                            salinity=salinity, #either 'salinity' or 'brine_volume_fraction' should be given for sea ice; if salinity is given, brine volume fraction is calculated in the model; if none is given, ice is treated as fresh water ice 
                            porosity = porosity, # either density or 'porosity' should be set for sea ice. If porosity is given, density is calculated in the model. If none is given, ice is treated as having a porosity of 0% (no air inclusions)
                            corr_length=p_ex,
                            add_water_substrate="ocean" #see comment below
                            )

a = 2.9
b = -782

# Import a .PRO file

In [4]:
test_filename = 'profile_example.pro'

variables_dict = utils.get_pro_raw(test_filename)

series_length = len(variables_dict['Date'])

In [15]:
my_stickiness = 0.2

my_evolver = evolving_pack()

for entrynum in trange(1,series_length,350):

    my_snowpro = utils.snowpro_from_snapshot(entrynum,variables_dict)

    my_snowpro.smrt_res = run_SMRT_on_snap(my_snowpro.iceframe,
                                                     my_snowpro.snowframe,
                                                     shs_factor=1/2,
                                                     stickiness = my_stickiness,
                                                     brine_inc_corr_len=1.0e-3,
                                                     solver='dmrt_qms')

#     my_evolver.update_pack(my_snowpro)


# plt.plot(my_evolver.datetime_series, my_evolver.Comiso_SD_series, label=str(my_stickiness))


# plt.xticks(rotation=90)
# plt.legend()
# plt.show()

100%|██████████| 5/5 [00:00<00:00, 89.92it/s]

qms
ERROR
qms
ERROR
qms
ERROR
qms
ERROR
qms
ERROR


In [14]:
def run_SMRT_on_snap(ice_df,
                         snow_df,
                         shs_factor=1/2,
                         stickiness = 0.1,
                         brine_inc_corr_len=1.0e-3,
                         solver='dort'):

    snow_grain_radius_m = snow_df['grain size (mm)'] * 0.5 * 1e-3 * shs_factor

    snow_df['radius_m'] = snow_grain_radius_m

    snowpack = make_snowpack(thickness=snow_df['thickness_m'],

                             microstructure_model="sticky_hard_spheres",

                             density=snow_df['element density (kg m-3)'],

                             temperature=snow_df['element temperature (degC)'] + 273.15,

                             radius=snow_df['radius_m'],

                             stickiness=stickiness)

    ice_col = make_ice_column(ice_type='firstyear',
                              thickness=ice_df['thickness_m'],
                              temperature=ice_df['element temperature (degC)'] + 273.15,
                              microstructure_model='exponential',
                              corr_length=np.array([brine_inc_corr_len] * len(ice_df['thickness_m'])),
                              brine_inclusion_shape='spheres',
                              salinity=ice_df['bulk salinity (g/kg)'] / 1000,
                              density=ice_df['element density (kg m-3)'],
                              add_water_substrate='ocean')

    medium = snowpack + ice_col

    # create the sensor
    radiometer = sensor_list.passive([37e9, 19e9], 55, "V")

    n_max_stream = 64  # TB calculation is more accurate if number of streams is increased (currently: default = 32);

    try:
        if solver == 'dort':
            m = make_model("iba", solver,
                           rtsolver_options={"n_max_stream": n_max_stream})

            # run the model for snow-covered sea ice:
            res = m.run(radiometer, medium)
        elif solver == 'dmrt_qms':
            print('qms')
            res = dmrt_qms_legacy.run(radiometer,
                                      medium,
                                      dmrt_qms_path = '/home/robbie/DMRT-QMS/')
    except:
        if solver == 'dort':
            print('WARNING: Renormalisation failed - forcing...')

            m = make_model("iba", solver,
                           rtsolver_options={"n_max_stream": n_max_stream,
                                             "phase_normalization": 'forced'})

            # run the model for snow-covered sea ice:
            res = m.run(radiometer, medium)
        else:
            print('ERROR')
            return(0)

    return (res)

In [ ]:
test_filename = 'profile_example.pro'

variables_dict = utils.get_pro_raw(test_filename)

series_length = len(variables_dict['Date'])

for my_shs in np.arange(0.1,0.5,0.05):
    
    print(my_shs)
    
    my_evolver = evolving_pack()

    for entrynum in trange(1,series_length,350):

        my_snowpro = utils.snowpro_from_snapshot(entrynum,variables_dict)

        my_snowpro.smrt_res = utils.run_SMRT_on_snapshot(my_snowpro.iceframe,
                                                         my_snowpro.snowframe,
                                                         shs_factor=my_shs,
                                                         stickiness = 0.15,
                                                         brine_inc_corr_len=1.0e-3)

        my_evolver.update_pack(my_snowpro)
        
    plt.plot(my_evolver.datetime_series, my_evolver.Comiso_SD_series, label=str(rd2(my_shs)))

plt.plot(my_evolver.datetime_series, my_evolver.snow_depth_series, label='real')

plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
import utils
from tqdm import trange
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
from classes import evolving_pack

test_filename = 'profile_example.pro'

variables_dict = utils.get_pro_raw(test_filename)

series_length = len(variables_dict['Date'])

for my_var in np.arange(0.5,2,0.2):
    
    print(my_var)
    
    my_evolver = evolving_pack()

    for entrynum in trange(1,series_length,350):

        my_snowpro = utils.snowpro_from_snapshot(entrynum,variables_dict)

        my_snowpro.smrt_res = utils.run_SMRT_on_snapshot(my_snowpro.iceframe,
                                                         my_snowpro.snowframe,
                                                         shs_factor=0.4,
                                                         stickiness = 0.2,
                                                         brine_inc_corr_len=my_var*1e-3)

        my_evolver.update_pack(my_snowpro)
        
    plt.plot(my_evolver.datetime_series, my_evolver.Comiso_SD_series, label=str(rd2(my_var)))

plt.plot(my_evolver.datetime_series, my_evolver.snow_depth_series, label='real')

plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
import utils
from tqdm import trange
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
from classes import evolving_pack

test_filename = 'profile_example.pro'

variables_dict = utils.get_pro_raw(test_filename)

series_length = len(variables_dict['Date'])

my_evolver = evolving_pack()

for entrynum in trange(1,series_length,100):

    my_snowpro = utils.snowpro_from_snapshot(entrynum,variables_dict)

    my_snowpro.smrt_res = utils.run_SMRT_on_snapshot(my_snowpro.iceframe,
                                                     my_snowpro.snowframe,
                                                     shs_factor=0.4,
                                                     stickiness = 0.2,
                                                     brine_inc_corr_len=1e-3)

    my_evolver.update_pack(my_snowpro)

plt.plot(my_evolver.datetime_series, my_evolver.Comiso_SD_series, label=str(my_var))

plt.plot(my_evolver.datetime_series, my_evolver.snow_depth_series, label='real')

plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:

def rd2(num):
    return (np.round(num, decimals=2))

NameError: name 'np' is not defined

# Can we make SMRT predict the right gradient ratio for a given snow depth?

In [ ]:
GR_SD = []
Re_SD = []

thickness, density, temperature, radius = [0.05],[300],[265],[1e-3]

for t in range(5):
    
    total_thickness = np.sum(thickness)

    snowpack = make_snowpack(thickness=thickness,

                                 microstructure_model="sticky_hard_spheres",

                                 density=density,

                                 temperature=temperature,

                                 radius=radius,

                                 stickiness=0.1)

    medium = snowpack + ice_column

    radiometer = sensor_list.passive([37e9, 19e9], 55, "V")

    n_max_stream = 64  

    m = make_model("iba", "dort",
                   rtsolver_options={"n_max_stream": n_max_stream})

    res = m.run(radiometer, medium)
    
    TbV_37 = float(res.TbV()[0])
    TbV_19 = float(res.TbV()[1])
    GR = (TbV_37-TbV_19)/(TbV_37+TbV_19)

    SD = (a+(b*GR))/100

    GR_SD.append(rd2(SD))
    Re_SD.append(total_thickness)
                 
    thickness.append(5)
    density.append(300)
    temperature.append(265)
    radius.append(1e-3)
    
    print(thickness,density,temperature,radius)


plt.plot(Re_SD,GR_SD)